## M5 evaluation

In [ ]:
import pandas as pd

from datasetsforecast.m5 import M5, M5Evaluation

### Evaluator

In [ ]:
def evaluate_forecasts(df, model, model_name):
    Y_hat = df.set_index('ds', append=True)[model].unstack()
    *_, S_df = M5.load('data')
    Y_hat = S_df.merge(Y_hat, how='left', on=['unique_id'])
    eval_ = M5Evaluation.evaluate(y_hat=Y_hat, directory='./data')
    eval_ = eval_.rename(columns={'wrmsse': f'{model_name}_{model}_wrmsse'})
    return eval_

### Read data

In [ ]:
statsforecasts_df = pd.read_parquet('s3://m5-benchmarks/forecasts/statsforecast-m5.parquet')

In [ ]:
amazonforecasts_df = pd.read_parquet('s3://m5-benchmarks/forecasts/amazonforecast-m5.parquet')

In [ ]:
amazonforecasts_df = amazonforecasts_df.rename(columns={'item_id': 'unique_id', 'date': 'ds', 'mean': 'AutoPredictor'})
amazonforecasts_df['unique_id'] = amazonforecasts_df['unique_id'].str.upper()
amazonforecasts_df = amazonforecasts_df.set_index('unique_id')
amazonforecasts_df['AutoPredictor'] = amazonforecasts_df['AutoPredictor'].clip(0) 

### Evaluate performances

In [ ]:
pd.concat([
    evaluate_forecasts(statsforecasts_df, 'ThETS', 'StatsForecast'),
    evaluate_forecasts(amazonforecasts_df, 'AutoPredictor', 'AmazonForecast'),
    evaluate_forecasts(statsforecasts_df, 'SeasonalNaive', 'StatsForecast'),
], axis=1)

,StatsForecast_ThETS_wrmsse,AmazonForecast_AutoPredictor_wrmsse,StatsForecast_SeasonalNaive_wrmsse
Total,0.669606,1.616653,0.847017
Level1,0.424331,1.910585,0.560047
Level2,0.515777,1.785607,0.673050
Level3,0.580670,1.735073,0.717606
Level4,0.474098,1.970949,0.622975
Level5,0.552459,2.008716,0.708155
Level6,0.578092,1.804468,0.759991
Level7,0.651079,1.817839,0.829187
Level8,0.642446,1.665958,0.800835
Level9,0.725324,1.617998,0.888126
